In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1. first we will import what will we deel with


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
from nltk.corpus import stopwords
import re
import networkx
from textblob import TextBlob

import warnings
warnings.filterwarnings("ignore")

# 2. importing data

In [ ]:
data = pd.read_csv('/kaggle/input/pfizer-vaccine-tweets/vaccination_tweets.csv')
data.head(3)

getting some information about our data to check how many nulls and non nulls

In [ ]:
data.info()

# we will deel with text feature and doing analysis 

first we should clean our text by removing hash tags, hyperlink,.....etc

In [ ]:
#cleaning the tweets
def clean_tweets(tweet):
 
    tweet = re.sub(r':', '', str(tweet))
    tweet = re.sub(r'‚Ä¶', '', str(tweet))
    tweet = re.sub('@[A-Za-z0–9]+', '', str(tweet))               
    tweet = re.sub('#', '', str(tweet)) # Removing '#' hash tag
    tweet = re.sub('https?:\/\/\S+', '', str(tweet)) # Removing hyperlink
    #replace consecutive non-ASCII characters with a space
    tweet = re.sub(r'[^\x00-\x7F]+',' ',str(tweet))
    tweet = str(tweet).lower()
    tweet = re.sub('\[.*?\]', '', tweet)
    tweet = re.sub('https?://\S+|www\.\S+', '', tweet)
    tweet = re.sub('<.*?>+', '', tweet)
    tweet = re.sub('\n', '', tweet)
    tweet = re.sub('\w*\d\w*', '', tweet)
    tweet = re.sub(r'#','',tweet)
    tweet = re.sub(r'RT[\s]+','',tweet)
    tweet = re.sub(r'[^\w]', ' ', tweet)
    tweet = re.sub('https', '', tweet)
    tweet = re.sub('https ', '', tweet)
    tweet = re.sub('co', '', tweet)

    return tweet

calculating subjectivity for our sentimental analysiss

In [ ]:
## Sentiment Analysis 

def getSubjectivity(tweet):
    try:
        return TextBlob(tweet).sentiment.subjectivity
    except:
        return None

note that the polarity give us the evaluation of each tweet in numbers,the positive
number refer to positive sentiment ,and zero is neutral ,and the negative is refer to
negative sentimental)

In [ ]:
# Create a function to get the polarity
def getPolarity(tweet):
    try:
        return  TextBlob(tweet).sentiment.polarity
    except:
        return None

In [ ]:
def sentiment_calc(tweet):
    try:
        return TextBlob(tweet).sentiment
    except:
        return None

In [ ]:
data['cleaned_text']=data['text'].apply(clean_tweets)

# Create two new columns 'Subjectivity' & 'Polarity'

In [ ]:
data['subjectivity'] = data['cleaned_text'].apply(getSubjectivity)
data['polarity'] = data['cleaned_text'].apply(getPolarity)
data['sentiment'] = data['cleaned_text'].apply(sentiment_calc)


In [ ]:
def getAnalysis(score):
    if score < 0:
      return 'Negative'
    elif score == 0:
      return 'Neutral'
    else:
      return 'Positive'

In [ ]:
data['analysis'] = data['polarity'].apply(getAnalysis)

In [ ]:
data.head()

In [ ]:
data.groupby('analysis')['analysis'].count().plot.bar()
plt.title('Sentiment Analyisis  ')




In [ ]:
x=data.groupby('analysis')['analysis'].count()
neutral=data[data['analysis']=='Neutral']['cleaned_text'].count()
positive=data[data['analysis']=='Positive']['cleaned_text'].count()
negative=data[data['analysis']=='Negative']['cleaned_text'].count()


In [ ]:
import plotly.graph_objects as go

In [ ]:
fig = go.Figure(data=[go.Pie(labels=["positivity","negativity","neutrality"], values=[positive,negative,neutral])])
fig.update_layout(title_text='Sentiment analysis of Pfizer vaccine')
fig.show()


In [ ]:
neutral_tweets=data[data['analysis']=='Neutral']['cleaned_text']
positive_tweets=data[data['analysis']=='Positive']['cleaned_text']
negative_tweets=data[data['analysis']=='Negative']['cleaned_text']


In [ ]:
from wordcloud import WordCloud
fig, (ax1, ax2,ax3) = plt.subplots(1, 3, figsize=[20, 20])
wordcloud1 = WordCloud( background_color='white',
                        width=600,
                        height=400).generate(" ".join(neutral_tweets))
ax1.imshow(wordcloud1)
ax1.axis('off')
ax1.set_title('neutral_tweets',fontsize=40);

wordcloud2 = WordCloud( background_color='white',
                        width=600,
                        height=400).generate(" ".join(positive_tweets))
ax2.imshow(wordcloud2)
ax2.axis('off')
ax2.set_title('positive_tweets',fontsize=40);

wordcloud2 = WordCloud( background_color='white',
                        width=600,
                        height=400).generate(" ".join(negative_tweets))
ax3.imshow(wordcloud2)
ax3.axis('off')
ax3.set_title('negative_tweets',fontsize=40);